An Intro to Python Type Hinting, with Focus on VSCode
=====================================================

A psuedo-history
----------------
In the beginning was Java, and it was statically typed, and it was a PITA. So Python came and rejected static typing, and the type of variable could change from invocation to invocation, and it was good-ish. But as Python applications grew, developers started to miss the certainty which Java's static typing provided, so type hints were born.

What type hints do
------------------
- provide clues (hence 'hints') to the variable types expected by a function
- allow your IDE to highlight instances where you're passing a type which doesn't match what is expected
- show professionalism in your code
- help guide your thinking as you develop

What type hints don't do
------------------------
- perform validation[^1]
- change how your code behaves at run time

[^1]: though they do provide the framework for the Pydantic validation library, which DOES do validation

When should you use type hints
-----------------------------
- anytime you're developing a library or package which will be used by other people
- anytime you're application will be taking input from users or other applications
- ALWAYS, for points 3 and 4 of what type hints do



## Enabling type checking in VSCode
To access typechecking in VSCode, install the PyLance extension (default with Python extension) and use menu (Code-> Settings -> Settings) or Cmd+, to open VSCode settings. Then select the desired option under the Extensions > PyLance > Type Checking

Options:
- off - no highlighting of issues
- basic - if you've hinted and done them wrong, it will highlight it
- standard - doesn't seem to be any different from basic? :shrug:
- strict - all the above, plus really finicky highlighting of issues where you haven't hinted things

## Using type hinting in your code

In [1]:
def unhinted(var1, var2):
    return var1 * var2

unhinted(3, 3)

9

In [9]:
unhinted(3, 'bar')

'barbarbar'

### Functions
To add type hinting to the function, specify the type for each argument and add `-> [return_type]` after the closing paren in the `def` line.

In [3]:
def hinted(var1: int, var2: int):
    return var1 * var2

hinted(3, 3)

9

In [4]:
#
# Notice how the behavior of this method doesn't change, but `bar` is now highlighted in the editor as incorrect.
#
hinted(3, 'bar')

'barbarbar'

#### Type hinting parameters with default values

When type hinting parameters with default values, the pattern is to put the type between the varname and the default declation, like: `{varname}: {type} = {default_value}`

In [11]:
def hinted_default(var1: int, var2: int = 3):
    return var1 * var2

hinted_default(4, 6)

24

In [12]:
hinted_default(4)

12

#### Return type hints 

In [5]:
#
# When type checking is shown, this will flag an issue with `foo`, because it's not an int.
#
def badreturn_hinted(var1: int, var2: int) -> int:
    return 'foo'

#
# Gotcha! This won't flag an issue because booleans are compatible with ints.
#
def gotcha(var1: int, var2: int) -> int:
    return False

assert False == 0

### Hinting complex types

Complex types like lists, dicts, tuples, or sets follow a more detailed notation.

In [2]:
#
# You can specify just the main type and not its content types.
#
def take_generic_list(mylist: list) -> None:
    pass

take_generic_list([1, 2, 3, 4.5, 'bar'])

#
# More robust hinting would also declare the content expected within the base types.
#
def take_list(mylist: list[int]) -> None:
    """
    This expects a variable like:
    [1,2,3]
    """
    pass

# This is highlighted because 4.5 and 'bar' are not int compatible.
take_list([1, 2, 3, 4.5, 'bar'])

def take_dict(mydict: dict[int, str]) -> None:
    """
    This expects a variable like:
    {
        1: 'Foo',
        2: 'Bar'
    }
    """
    pass

take_dict({1: 'foo', 2: 'bar', 3: 42})

#
# Nested objects are specified just like others.
#
def take_nested_dict(nested_dict: dict[int, dict[str, str]]) -> None:
    """
    This will expect something like:
    {
        1: {
            'name': 'Lancelot',
            'favorite_color': 'Blue'
        }
    }
    """
    pass

grail_knights = {
        42: {
            'name': 'Galahad',
            'favorite_color': 'Blue. No yel-- Auuuuuuuugh!'
        }
    }
take_nested_dict(grail_knights)

### Hinting to allow multiple types

The pipe operator (`|`) allows you declare a parameter as handling multiple types.

In [ ]:
def multi_types(var1: int | float, var2: int | float) -> None:
    pass

multi_types(3, 9)

multi_types(3, 1.5)

### Hinting classes

In [ ]:
class Foo():
    pass

class Bar(Foo):
    pass

class Fnord():
    pass

def handle_foo(my_foo: Foo) -> bool:
    return True

foo = Foo()
bar = Bar()
fnord = Fnord()

# hinting works with classes, even through inheritance
handle_foo(foo)
handle_foo(bar)
handle_foo(fnord)

# hinting in this instance will NOT work UNLESS you have type checking set to `strict`
for v in [foo, bar, fnord]:
    handle_foo(v)

## The Any type

There could eventually be a time in which the variable you're defining will accept any variable type that someone wants to throw at it. This _should_ be very, very rare, but if it happens, the Any type is the type hint for you.

You should regard as a form of code smell and make sure it's appropriate and not just someone being lazy, but it's there if you have a need for it.

In [6]:
from typing import Any

def my_indiscriminate_function(input_var: Any) -> Any:
    print(f'You handed me a variable of type {type(input_var)}. Here it is back.')
    return input_var

my_indiscriminate_function(9999)
my_indiscriminate_function('foo')
my_indiscriminate_function(gotcha)

You handed me a variable of type <class 'int'>. Here it is back.
You handed me a variable of type <class 'str'>. Here it is back.
You handed me a variable of type <class 'function'>. Here it is back.


<function __main__.gotcha(var1: int, var2: int) -> int>

## Type checking outside the IDE: the mypy package

The [mypy package](https://mypy.readthedocs.io/en/stable/index.html) is installable with pip and lets you do type checking of a file from the command line. This means you could use it as part of a CI workflow or in a githook to ensure your code is type correct.

> $ mypy program.py